# Alternative Traveling Salesman Algorithm
### (Still a work in progress)

## I was inspired by the innate optimization of nature. I imagined a cross-section of a bubble enveloping an object whose vertices make up the nodes of a traveling salesman graph. This technique is a modified greedy algorithm starting from all points on the convex hull, where one is trying to minimize the concavity of the convex hull. 

## The primary differences between this and a typical greedy algorithm are that all boundary points of the convex hull are starting points and that edges of the "bubble" merge.

### Pseudocode:

1. Start with a convex hull whose points are OP and with the interior points, IP.<br /><br />
2. Find Euclidean distances between every pair of points.<br /><br />
3. The shortest distance for any OP to an IP is the current outer radius, "OR".<br /><br />
4. WHILE all IP are NOT connected by two edges,<br /><br />
&nbsp;&nbsp;&nbsp;&nbsp;a) IF the distance from that now connected IP, namely IP_1, is longer than any <br />
distance from IP_1 to another unconnected IP or another edge (as edges can <br />
connect to each other)<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i) THEN connect to that point AND repeat step 4.<br /><br />
&nbsp;&nbsp;&nbsp;&nbsp;b) ELSE, all newly connected IP_n become part of the OP set, AND CONTINUE to<br />
next smallest "OR" (including the connected IP_n).<br /><br />
&nbsp;&nbsp;&nbsp;&nbsp;c) Whenever inner edges from two OP, through any number of IP_n, connect, DELETE <br />
the edge between those OP (including IP_n that are now OP).

## Another visual analogy is to imagine a vacuum seal bag around an irregular buckyball. Take a cross section of that object where the center of mass is in the cross section. With this algorithm, I am attempting to represent the effect of sucking out the air from the center of mass of the buckyball. 

## The significance of this, I believe, is that the edges can connect and that connections are made to the nearest connected nodes in ascending order (a kind of greedy algorithm from every point on the complex hull). With similar algorithms, I believe that either all edges are measured relative to the start or that they do not connect on the edge too.

# I believe that this algorithm doesn't work in many cases, but I am unsure which cases it does work for. The questions that I seek to answer are:
### What is the time complexity of this algorithm as is, regardless of the conditions of the graph (especially given that the convex hull needs to be found first), as it may be that it isn't significant  regardless of the graph type?<br /><br />What types of graphs does this algorithm work for? This is where my inspiration from natural physical optimization and the "vacuum effect" come from.<br /><br />Furthermore, does the "vacuum effect" represent some geometric center where if this effect occurs within the ultimate optimal path, then the algorithm may work?<br /><br />Finally, what is the probability that this "center" generally falls within the optimal path, and can a Monte Carlo simulation be used to answer this question?

In [1]:
from scipy.spatial import ConvexHull
import numpy as np
from math import sqrt
from itertools import combinations

In [2]:
allPoints = []

n=15 # number of points
N=10 # +/- range of points 

for i in range(n):
    points = (float(np.random.randint(-N,N)),float(np.random.randint(-N,N)))
    allPoints.append(points)
# creates list of random points

allPoints = list(set(allPoints)) #takes out possible duplicates

print("allPoints:",allPoints,"\n")

allPoints: [(3.0, -8.0), (-8.0, -2.0), (9.0, 0.0), (2.0, 6.0), (6.0, 7.0), (-8.0, 7.0), (-6.0, 0.0), (3.0, -9.0), (-2.0, -1.0), (3.0, -1.0), (2.0, 0.0), (-5.0, 5.0), (7.0, -3.0), (-9.0, 1.0), (0.0, -4.0)] 



In [3]:
class TravelPoly:
    def __init__(self,cls):
        self.OP = [allPoints[i] for i in ConvexHull(allPoints).vertices]
        print("OP:",self.OP,"\n")
        
        self.IP = [x for x in allPoints if x not in self.OP]
        print("IP:",self.IP,"\n"*2)

        self.currEdges = []
        self.outerDistsDict = {}
        self.innerDistsList = []

        self.touchedIP = []

        cls.getCurrEdges(self,cls)
        cls.getOuterDistsDict(self,cls)
        cls.getInnerDistsList(self,cls)

    def algorithm(self,cls):
        '''
        TODO WHILE there are IP (inner points) not connected by two edges
        '''
        for i in range(10): #TODO replace FOR with WHILE loop 
            cls.getMinOR(self,cls)
            self.listIR = self.listOR
            
            cls.getListIR(self,cls,self.listOR)
            self.listIR.sort(key=lambda tup: tup[1][1])
            print("listIR:",self.listIR)
            
            cls.getListIR(self,cls,self.listIR)
            self.listIR.sort(key=lambda tup: tup[1][1])
            print("listIR:",self.listIR)

            iterList = self.listIR.copy()
            self.listIR = []
            cls.upThroughIR(self,cls,iterList) 
                
    @staticmethod
    def distance(point1, point2):
        return sqrt((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)    
        
    def getCurrEdges(self,cls):    
        for i in range(len(self.OP)):
            self.currEdges = [(self.OP[i],self.OP[(i+1)%len(self.OP)]) for i in range(len(self.OP))] 
        print("currEdges:",self.currEdges,"\n"*2)

    def getOuterDistsDict(self,cls): 
        print("outerDistsDict:")
        for a in self.OP:
            temp = []
            for b in self.IP:
                temp.append((b,cls.distance(a,b)))
            
            temp.sort(key=lambda tup: tup[1])
            print(a,":",temp,"\n")
            self.outerDistsDict.update({a:temp})
            
    def getInnerDistsList(self,cls):
        print("\n""innerDistsList:")
        for a,b in combinations(self.IP,2):
            self.innerDistsList.append(((a,b),cls.distance(a,b)))
        
        self.innerDistsList.sort(key=lambda tup: tup[1])
        print(self.innerDistsList)

    def getMinOR(self,cls):
        self.listOR = [] 
        min_value = float("inf")
        for k,v in self.outerDistsDict.items():
            if v[0][1] == min_value:
                cls.checkForMultiOR(self,k,v,min_value,self.listOR)
            elif v[0][1] < min_value:
                min_value = v[0][1]
                self.listOR = []
                cls.checkForMultiOR(self,k,v,min_value,self.listOR)
        
        print("\n"*2,"listOR:",self.listOR)
        for key,val in self.listOR: # Removes minOR_list values from outerDistsDict dictionary
            self.outerDistsDict[key].remove(val)

    def checkForMultiOR(self,key,val,min_value,radiusList):
        radiusList.append((key,(val[0][0],val[0][1])))
        for i in range(len(val)-1):
            if val[i+1][1] == min_value:
                radiusList.append((key,(val[i+1][0],val[i+1][1])))
            else:
                break
        
    def checkForIR(self,radiusList):       
        temp = []
        for (i,j),k in self.innerDistsList:
            for a,(b,c) in radiusList:
                if k <= c:
                    if b == j:
                        if not temp:
                            temp.append((b,(i,k)))
                        elif k == temp[0][1][1]:
                            temp.append((b,(i,k)))
                        elif k < temp[0][1][1]:
                            temp = []
                            temp.append((b,(i,k)))
                    elif b == i:
                        if not temp:
                            temp.append((b,(j,k)))
                        elif k == temp[0][1][1]:
                            temp.append((b,(j,k)))
                        elif k < temp[0][1][1]:
                            temp = []
                            temp.append((b,(j,k)))
                else:
                    return temp                  

    def getListIR(self,cls,radiusList):
        temp = cls.checkForIR(self,radiusList)
        self.listIR += temp  
        print("temp:",temp,"\n")
        for a,(b,c) in temp: # Removes temp values from innerDistsList dictionary
            try:
                self.innerDistsList.remove(((a,b),c))
            except:
                try:
                    self.innerDistsList.remove(((b,a),c))
                except:
                    
                    
        
        if temp:
            cls.getListIR(self,cls,temp)

    def upThroughIR(self,cls,iterList):
        del iterList[0]
        if iterList:
            cls.getListIR(self,cls,iterList)
            iterList += self.listIR
            iterList.sort(key=lambda tup: tup[1][1])
            self.listIR = []
            print("iterList:",iterList)
            cls.upThroughIR(self,cls,iterList)   

    def updateOP(self,listName):
        #for a,(b,c) in listName:
        pass
    
    def getTouchedPoints(self):
        #for val in self.listIR:
        pass

    def updateEdges(self):
        #for val in self.listIR:
        pass

    def breakEdges(self):
        pass

In [4]:
TravelPoly(TravelPoly).algorithm(TravelPoly)

OP: [(3.0, -9.0), (9.0, 0.0), (6.0, 7.0), (-8.0, 7.0), (-9.0, 1.0), (-8.0, -2.0)] 

IP: [(3.0, -8.0), (2.0, 6.0), (-6.0, 0.0), (-2.0, -1.0), (3.0, -1.0), (2.0, 0.0), (-5.0, 5.0), (7.0, -3.0), (0.0, -4.0)] 


currEdges: [((3.0, -9.0), (9.0, 0.0)), ((9.0, 0.0), (6.0, 7.0)), ((6.0, 7.0), (-8.0, 7.0)), ((-8.0, 7.0), (-9.0, 1.0)), ((-9.0, 1.0), (-8.0, -2.0)), ((-8.0, -2.0), (3.0, -9.0))] 


outerDistsDict:
(3.0, -9.0) : [((3.0, -8.0), 1.0), ((0.0, -4.0), 5.830951894845301), ((7.0, -3.0), 7.211102550927978), ((3.0, -1.0), 8.0), ((2.0, 0.0), 9.055385138137417), ((-2.0, -1.0), 9.433981132056603), ((-6.0, 0.0), 12.727922061357855), ((2.0, 6.0), 15.033296378372908), ((-5.0, 5.0), 16.1245154965971)] 

(9.0, 0.0) : [((7.0, -3.0), 3.605551275463989), ((3.0, -1.0), 6.082762530298219), ((2.0, 0.0), 7.0), ((2.0, 6.0), 9.219544457292887), ((0.0, -4.0), 9.848857801796104), ((3.0, -8.0), 10.0), ((-2.0, -1.0), 11.045361017187261), ((-5.0, 5.0), 14.866068747318506), ((-6.0, 0.0), 15.0)] 

(6.0, 7.0) : [((2

ValueError: list.remove(x): x not in list